## Import Libraries, Initialize Models, etc


In [66]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv

from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

import pandas as pd
import os

load_dotenv("../.env")

True

In [67]:
# model_name = "BAAI/bge-m3"

# embedding_model = HuggingFaceInferenceAPIEmbeddings(
#     api_key=os.getenv("HUGGINGFACE_API_KEY"), model_name=model_name
# )
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# # test if embedding model works
# res = embedding_model.embed_query("The quick brown fox jumps over the lazy dog")
# res[:3]

## Load Sheets


In [68]:
def read_dfs(sheet_names: list[str]) -> dict[str, pd.DataFrame]:
    # filepath = "xlsx/[ANNOTATED] Informasi Umum ITB.xlsx"
    # dfs = pd.read_excel(filepath, sheet_name=sheet_names)
    url = "https://docs.google.com/spreadsheets/d/1p0_lwcGKLP5NtsLx_cd5kCyc7i58VuTV/edit?usp=sharing&ouid=103581786644820929582&rtpof=true&sd=true"
    url_pandas = url.replace("/edit?usp=sharing", "/export?format=xlsx")
    dfs = pd.read_excel(url_pandas, sheet_name=sheet_names)

    for name in sheet_names:
        # for each df, drop a row if the entire row is null
        # also drop cells with the content "kembali ke halaman utama"
        dfs[name] = dfs[name].dropna(how="all").reset_index(drop=True)
        dfs[name] = dfs[name][
            ~dfs[name].iloc[:, 0].str.contains("Kembali ke Halaman Utama", na=False)
        ]

        # # do not make the first row as the title of the column
        # dfs[name].columns = range(dfs[name].shape[1])

    return dfs

## Process Sheets


In [78]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 0

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    is_separator_regex=False,
)


def process_standard_sheets(sheet_name: str, df: pd.DataFrame):
    # we want to read cell by cell: there is only one column in this dataframe
    # essentially, we want to group the content between cells with the tag [H1]
    # get the indices of the cells with the tag [H1]
    documents = []
    h1_indices = df[df.iloc[:, 0].str.contains("\[H1\]")].index
    ref = df.columns[0].split("Referensi: ", 1)[1]

    for i in range(len(h1_indices)):
        index = h1_indices[i]
        next_idx = h1_indices[i + 1] if i + 1 < len(h1_indices) else len(df)

        # get the title and the content of the part between the [H1] tags
        title = df.iloc[index, 0]
        content = "".join(str(df.iloc[index + 1 : next_idx, 0].values))
        content = content.replace("\\xa0", " ").replace("\n", "")

        title = title.replace("[H1] ", "")

        texts = text_splitter.create_documents([content])

        for i in range(len(texts)):
            texts[i].metadata = {
                "title": title,
                "sheet_name": sheet_name,
                "reference": ref,
            }
            texts[i].page_content = f"{texts[i].page_content} - Reference: {ref}"

        documents.extend(texts)

    return documents


def process_sheets_with_tables(sheet_name: str, df: pd.DataFrame):
    documents = []
    h1_indices = df[df.iloc[:, 0].str.contains("\[H1\]", na=False)].index
    table_indices = df[df.iloc[:, 0].str.contains("\[TABLE\]", na=False)].index
    ref = df.columns[0].split("Referensi: ", 1)[1]

    for i in range(len(h1_indices)):
        index = h1_indices[i]
        next_idx = h1_indices[i + 1] if i + 1 < len(h1_indices) else len(df)

        # find the table index that is more than index but less than next idx
        table_index = table_indices[
            (table_indices > index) & (table_indices < next_idx)
        ]

        # update the next_idx to be the table index if it exists
        if len(table_index) > 0:
            next_idx = table_index[0]

        # get the title and the content of the part between the [H1] tags
        title = df.iloc[index, 0]
        content = "".join(str(df.iloc[index + 1 : next_idx, 0].values))
        content = content.replace("\\xa0", " ").replace("\n", "")

        title = title.replace("[H1] ", "")

        texts = text_splitter.create_documents([content])

        for i in range(len(texts)):
            texts[i].metadata = {"title": title, "sheet_name": sheet_name}
            texts[i].page_content = f"{texts[i].page_content} - Reference: {ref}"

        documents.extend(texts)

    return documents

## Process Tables


In [70]:
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader


def read_tables():
    url = "https://docs.google.com/spreadsheets/d/15nTfetJP_EO4uN7ATwVTPti12v_DppC4O49zbxgM1PE/edit?usp=sharing"
    url_pandas = url.replace("/edit?usp=sharing", "/export?format=xlsx")
    xls = pd.ExcelFile(url_pandas)

    dfs = {}
    for sheet_name in xls.sheet_names:
        df = xls.parse(sheet_name)
        dfs[sheet_name] = df

    print("total dfs:", len(dfs))
    return dfs

In [84]:
import os
from groq import Groq


PARSE_TABLE_SYSTEM_PROMPT = """
You are an agent specializing in converting a list of JSON representations into paragraphs of text in order to make it readable.
Avoid converting it into a list; prefer converting the representation into a readable sentence or paragraph.
You may segment the paragraphs to what you see fit. Only respond with the representation. 
"""
PARSE_TABLE_USER_PROMPT = """
Please convert this JSON representation into paragraphs of text:
{jsons}
"""

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))


def parse_tables(df: pd.DataFrame, sheet_name: str):
    ref = df.columns[0].split("Referensi: ", 1)[1]
    jsons = []
    docs = []

    # replace the header with the first row
    df.columns = df.iloc[0]
    df = df[1:]

    # convert this into a list of json, where each key is the column and each value is the value
    for i in range(len(df)):
        row = df.iloc[i]
        text = row.to_json()
        text = text.replace("\\xa0", " ").replace("\n", "")
        jsons.append(text)

    user_prompt = PARSE_TABLE_USER_PROMPT.format(jsons=jsons)
    completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": PARSE_TABLE_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0,
        model="llama-3.1-8b-instant",
    )

    content = completion.choices[0].message.content

    texts = text_splitter.create_documents([content])

    for i in range(len(texts)):
        texts[i].metadata = {"title": sheet_name, "sheet_name": sheet_name}
        texts[i].page_content = f"{texts[i].page_content} - Reference: {ref}"

    docs.extend(texts)

    return docs

In [85]:
all_documents = []


# standard sheets: readable ones with one column
standard_sheets = [
    "Tentang ITB",
    "Penerimaan",
    "Pendidikan",
    "Penelitian",
    "Pengabdian",
    "Multikampus",
    "FAQ Umum",
    "FAQ Beasiswa",
    "FAQ Penerimaan Mahasiswa Baru",
    "FAQ Teknik Fisika",
    "FAQ Lain-lain",
]

sheets_with_tables = [
    "Info Pendaftaran - S1 - Cleaned",
    "Info Pendaftaran - S2",
    "Info Pendaftaran - S3",
    "International Program - S1",
    "International Program - S2",
    "International Program - S3",
    "International Program - Student",
    "International Program - Scholar",
    "Program Profesi - Keinsinyuran",
    "Program Profesi - Apoteker",
    "Program Nonreguler - NRNG",
    "Program Nonreguler - MBKM ITB X",
    "Program Nonreguler - Summer Cou",
]


# read standard sheets
standard_dfs = read_dfs(standard_sheets)
sheets_with_tables_dfs = read_dfs(sheets_with_tables)
table_dfs = read_tables()

for sheet_name, df in standard_dfs.items():
    docs = process_standard_sheets(sheet_name, df)
    all_documents.extend(docs)

for sheet_name, df in sheets_with_tables_dfs.items():
    docs = process_sheets_with_tables(sheet_name, df)
    all_documents.extend(docs)

for sheet_name, df in table_dfs.items():
    docs = parse_tables(df, sheet_name)
    all_documents.extend(docs)


len(all_documents)

total dfs: 23


507

In [86]:
all_documents[-5:]

[Document(metadata={'title': 'ITB IUP Program Sched', 'sheet_name': 'ITB IUP Program Sched'}, page_content='The ITB International Undergraduate Program has several key dates to note. Online registration for the program will take place, with the first period scheduled from 13 February to 28 March 2024 and the second period from 30 April to 23 June 2024.\n\nThe ITB ELQ/AQ Test Simulation will be conducted on two separate dates. The first test will take place on 1 April 2024 from 13.00-14.00 WIB (UTC+7), while the second test will be held on 24 June 2024 from 13.00-14.15 WIB (UTC+7).\n\nThe ITB English Language Qualification Test (ITB ELQ Test) will be administered on two different dates. The first test will take place on 2 April 2024 from 13.00-15.00 WIB (UTC+7), while the second test will be held on 27 June 2024 from 13.00-17.00 WIB (UTC+7).\n\nThe ITB Academic Qualification Test (ITB AQ Test) will also be conducted on two separate dates. The first test will take place on 4 April 2024 f

## Embedding and Inserting to Vector Database


In [87]:
input_texts = [doc.page_content for doc in all_documents]

# embeddings = [embedding_model.embed_query(text) for text in input_texts]
embeddings = embedding_model.embed_documents(input_texts)

# embeddings = []

# for i, text in enumerate(input_texts):
#     print(f"Processing document {i+1}/{len(input_texts)}")
#     embeddings.append(embedding_model.embed_query(text))

In [88]:
client = QdrantClient(
    os.getenv("VECTOR_DB_ENDPOINT"),
    api_key=os.getenv("QDRANT_API_KEY"),
    prefer_grpc=True,
)
# client = QdrantClient(os.getenv("LOCAL_VECTOR_DB_ENDPOINT"))


client.recreate_collection(
    collection_name="informasi_umum_json_refs",
    vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE),
)

client.upsert(
    collection_name="informasi_umum_json_refs",
    points=[
        PointStruct(
            id=i,
            vector=vector,
            payload={
                "page_content": all_documents[i].page_content,
                "metadata": all_documents[i].metadata,
            },
        )
        for i, vector in enumerate(embeddings)
    ],
)

/tmp/ipykernel_20391/2665297733.py:9: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)